In [ ]:
#loading the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline



In [2]:
#Creating a function to load the text and labels from train and test set

In [3]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

In [4]:
train_labels[0]

1

In [5]:
train_texts[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [80]:
train_texts.info

AttributeError: 'list' object has no attribute 'info'

In [7]:
train_labels=train_labels[0:500]

In [8]:
train_texts=train_texts[0:500]

In [9]:
#text pre-processing

In [10]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)# removing non alphanumeric
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)# removing non ascii
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [11]:
train_texts[0]

'stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen    '

In [12]:
#countvectorizer
# convert text to numeric characters called count vectors

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)# converting text to 0, 1
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [72]:
l=['good is boy','bad is is boy boy boy' , 'someone is is feeling sleepy','sleepy' ,'sleep', 'feeling' , 'someone' , 'is' , 'was']

In [73]:
l_t= cv.transform(l)
# (0,2678) - is
# (0,600) - boy
# (0,1858) - feeling
#sleepy got ignored why? but sleep didnt get ignored
# why are we not removing is , was etc , they dont have any meaning right
# the algorithm reads 'the product is not good' same as 'good product is not the'

-----
# for training the model we can use the already labeled amazon reviews data from kaggle or 
# we can label all the 5 star comments as 1 , and all the 1 star comments as 0 and feed the model.

# after the training the model, it kind of creates a bag of positive words and a bag of negative words
# and when a new sentence is given the model sums up like +1.5 +1.7 -2.9 +4.2 +0.7  -2.2 = >0 marks it as a positive review
# the above score goes to a sigmoid function and gives out the probability ,we predict the probability
review example:
the product was good and everything else was awesome - +ve review , high probability
the product was good and delivery was ok - not sure +ve or -ve , +ve but low probability
probability 0 - 0.5 negative review
probaiility 0.5 to 1 positive review

stopwords - words that are to be removed 
stemming - interaction becomes interact , sleepy becomes sleep
lemmatization - better becomes good, ran , running becomes run
word_tokenize - 
logistic regression
multinomialNB
SVC
https://www.youtube.com/watch?v=lMQzEk5vht4
https://www.youtube.com/watch?v=o1Bb7G4szQQ

In [75]:
for i in l_t[4]:
    print(i)
    

  (0, 4610)	1


In [19]:
test_texts[0]

'great cd  my lovely pat has one of the great voices of her generation  i have listened to this cd for years and i still love it  when i m in a good mood it makes me feel better  a bad mood just evaporates like sugar in the rain  this cd just oozes life  vocals are jusat stuunning and lyrics just kill  one of life s hidden gems  this is a desert isle cd in my book  why she never made it big is just beyond me  everytime i play this  no matter black  white  young  old  male  female everybody says one thing  who was that singing   '

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(
    X, train_labels, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    # c called , how much do we want to regularize the model on to the data ,called the inverse strength of the regularisation
    #means what ?
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

C:\Users\BHARANI\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.712
Accuracy for C=0.05: 0.768
Accuracy for C=0.25: 0.752
Accuracy for C=0.5: 0.752
Accuracy for C=1: 0.76


In [18]:
lr.predict(X_test[29])

array([0])

In [19]:
test_labels[29]

0

In [22]:
test_texts[29]

'three days of use and it broke  very disappointed in this product  it worked perfectly for exactly three days and could not be resuscitated  it was very inexpensive so i did not want to pay half again the price to ship it back for an exchange  so the company would do nothing when they sent me an inquiry as to product satisfaction '